In [ ]:
# | echo: false
from nbdev import show_doc


In [ ]:
# | default_exp core


# core


> **O**bject **D**etection over **D**iscord using the Raspberry **PI**

## Setup

Required environment variables in `.env` file

- `PI_HOSTNAME`
    - Hostname (IP) to use for raspberry pi server
- `PI_PASSWORD`
    - to use `capture_and_fetch` on pi server
- `DISCORD_ID`
    - This is the Discord client ID token
- `DISCORD_TOKEN`
    - Discord bot token


## Camera and Network

Connect to Pi, take pictures, and save image locally

In [ ]:
# | export
import os

FILE_PATH = os.path.abspath("")


In [ ]:
# | eval: false
# | skip_showdoc
# | echo: false
FILE_PATH = (globals()["_dh"][0] / "odd_pi").as_posix()


In [ ]:
# | export
import discord
from pathlib import Path
import polars as pl
import paramiko
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from dotenv import load_dotenv
from rich import print
from cvu.detector.yolov5 import Yolov5 as Yolov5Onnx
from cvu.detector.prediction import Prediction
import asyncio
import datetime
from pathlib import Path

DATA_DIR = Path(FILE_PATH).parent.parent / "data"
# NAME_PATH = (DATA_DIR.parent / "models") / "yolo.names"
# TEST_PATH = DATA_DIR / "test.jpg"
CMD = "raspistill -t 0 -h 640 -w 640 -o ~/Desktop/capture.jpg"


In [ ]:
# | export
def pltimg(img: cv2.Mat) -> plt.Figure:
    """Plots picture"""
    return plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


def capture_and_fetch(
    username: str = "pi",  # Username for Raspberry Pi
    local_path: str = "./data/capture.jpg",  # Path to save image
    delay: int = None,  # Delay between command and picture is taken
    cmd=CMD,  # Raspberry pi camera command
):
    """Requires `PI_PASSWORD` be set in .env file. Delay is the number of milliseconds before taking picture"""
    load_dotenv()
    hostname = os.environ["PI_HOSTNAME"]
    password = os.environ["PI_PASSWORD"]
    assert hostname is not None, "Failed to get env_var `PI_HOSTNAME`"
    assert password is not None, "Failed to get env_var `PI_PASSWORD`"
    ssh_client = _connect(hostname, username, password)

    if delay:
        time.sleep(delay)

    ssh_client.exec_command(cmd)
    time.sleep(1)

    _fetch(ssh_client, "/home/pi/Desktop/capture.jpg", local_path)


def _connect(hostname: str, username: str, password: str) -> paramiko.SSHClient():
    """Connects to client"""
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(hostname=hostname, username=username, password=password)
    return ssh_client


def _fetch(client: paramiko.SSHClient, remote_path: str, local_path: str):
    """Fetches single file using SSHClient"""
    ftp_client = client.open_sftp()
    ftp_client.get(remote_path, local_path)
    ftp_client.close()


## Model and Prediction

Fetch YOLOv5 model and predict on image

In [ ]:
# | export
def predict(model: Yolov5Onnx, image_path: str) -> tuple[pl.DataFrame, np.ndarray]:
    """Runs model on input image and returns predictions and output image"""
    img = cv2.imread(str(image_path))
    preds = model(img)
    fig = pltimg(preds.draw(img))
    fig.write_png(str(image_path)[:-4] + "_yolo.png")
    return preds, img


def get_default_model() -> Yolov5Onnx:
    """Retrieves default YOLOv5 model"""
    return Yolov5Onnx(classes="coco", backend="onnx", weight="yolov5s", device="cpu")


def df_from_preds(preds: Prediction) -> pl.DataFrame:
    """Converts annoying object to DataFrame"""
    rows = []
    for row in str(preds).split("\n"):
        row = row.split("\t")
        row = [x.split("=")[1].strip(";") for x in row if "=" in x]
        rows.append(row)
    df = pl.DataFrame(rows)#.transpose()
    if df.is_empty():
        return None
    df = df.transpose()
    df.columns = ["class", "confidence", "tl", "br"]
    return df.select(["class", pl.col("confidence").cast(pl.Float32)])


Example use

```python
model = get_default_model()
preds, img = predict(model, '/path/to/img')
```

## Discord Bot

> Manage interactions to camera via Discord Bot

Type `python scripts/launch_discord_bot.py --help` in root directory for more information

In [ ]:
# | eval: false
# | echo: false
# Required to run in jupyter but not in our package
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# | export
def run_bot(
        run_active_cam: bool = True, # Runs camera at interval looking for a class to identify
        cam_interval: int = 30, # Number of seconds to camera checks for target class
        cam_class: str = 'cat', # Object to identify. Run `list_classes` to see all options
        cam_channel: str = 'general', # Disord channel to post active cam pictures to
        pic_keyword: str = 'pi!', # Command a user types in a Discord channel to take a picture
        yolo_keyword: str = 'yolo!', # Same as above but with the YOLO predictions overlayed
        pic_dir: str = './pics' # Intermediate directory for pictures
):
    """Run Discord bot that communicates with the Raspberry Pi camera""" 
    intents = discord.Intents.default()
    intents.message_content = True
    client = discord.Client(intents=intents)
    pic_dir = Path(pic_dir)
    pic_dir.mkdir(exist_ok=True)

    @client.event
    async def on_ready():
        """Indicates bot is ready"""
        print(f"{client.user} has connected to Discord. Hello!")

        # Cat Cam Coroutine
        channel = discord.utils.get(client.get_all_channels(), name=cam_channel)
        if run_active_cam:
            while True:
                await asyncio.sleep(cam_interval)
                await post_if_class(channel)

    @client.event
    async def on_member_join(member):
        """Responds to members joining"""
        await member.create_dm()
        await member.dm_channel.send(f"Hello and welcome {member.name}!")

    @client.event
    async def on_message(message):
        """Responds to messages from users"""
        if message.author == client.user:
            return

        elif message.content.lower() == pic_keyword:
            await take_picture(message)

        elif message.content.lower() == yolo_keyword:
            await take_and_model_picture(message)

    async def take_picture(message):
        """Takes picture and uploads it to channel"""
        msg = f"Sent picture to {message.channel.name} @ {message.created_at} triggered by {message.author}"
        capture_and_fetch(username="pi", local_path="./img.jpg")
        with open("./img.jpg", "rb") as f:
            picture = discord.File(f)
            print(msg)
            await message.channel.send(file=picture)

    async def take_and_model_picture(message):
        """Takes picture, runs YOLOv5, and uploads it to channel"""
        capture_and_fetch(username="pi", local_path="./img.jpg")
        model = get_default_model()
        preds, _ = predict(model, "./img.jpg")
        print(preds)
        await send_file(message, "./img_yolo.png")

    async def send_file(message, file_path):
        with open(file_path, "rb") as f:
            picture = discord.File(f)
            await message.channel.send(file=picture)

    async def post_if_class(channel: discord.TextChannel):
        pic_path = Path(pic_dir) / 'cam.jpg'
        capture_and_fetch(local_path=pic_path)
        model = get_default_model()
        preds, _ = predict(model, pic_path)
        preds = df_from_preds(preds)
        if preds is None:
            print('Nothing detected', datetime.datetime.now())
            return
        if cam_class.capitalize() in preds['class'].to_list():
            with open(Path(pic_dir) / 'cam_yolo.png', "rb") as f:
                picture = discord.File(f)
                print(f'{cam_class.capitalize()} detected!', datetime.datetime.now())
                print(preds)
                await channel.send(f'{cam_class.capitalize()} Cam!', file=picture)
        else:
            print(preds)
            print(f'{cam_class.capitalize()} not detected', datetime.datetime.now())

    load_dotenv()
    token = os.getenv("DISCORD_TOKEN")
    assert token is not None, "Failed to get env_var `DISCORD_TOKEN`"
    client.run(token)


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
